In [4]:
import pandas as pd
import numpy as np
import json
import os
import ast
from pathlib import Path
import torch
# from sentence_transformers import SentenceTransformer
from typing import List, Optional
from dataclasses import dataclass

import teradatasql
from sqlalchemy import create_engine
from dotenv import load_dotenv

from teradatagenai import VectorStore

os.makedirs('../results', exist_ok=True)


import sys
sys.path.append('..')
# from models import SentenceEmbeddingModel, SentenceEmbeddingConfig
# from utils import load_embedding_model
from constants import (
    CLEANED_TEST_DATA_PATH,
    ENCODED_TEST_DATA_PATH,
    CLEANED_TRAIN_DATA_PATH
)




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mk255155\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# @dataclass
# class SentenceEmbeddingConfig:
#     device: str
#     dtype: str
#     model_id: str
#     truncate_dim: Optional[int]
#     convert_to_numpy: bool
#     convert_to_tensor: bool

# class SentenceEmbeddingModel:
#     def __init__(self, config: SentenceEmbeddingConfig):
#         super().__init__()
#         self.config = config
#         self.model_id = config.model_id
#         self.device = config.device
#         self.dtype = config.dtype
#         self.truncate_dim = config.truncate_dim

#         self.model = SentenceTransformer(
#             self.model_id,
#             device=self.device,
#             truncate_dim=self.truncate_dim,
#             model_kwargs={"torch_dtype": getattr(torch, self.dtype) if isinstance(self.dtype, str) else self.dtype}
#         )

#     def get_embeddings(self, texts: List[str], prompt_name: Optional[str] = None):
#         embeddings = self.model.encode(
#             texts, 
#             prompt_name=prompt_name, 
#             convert_to_numpy=self.config.convert_to_numpy,
#             convert_to_tensor=self.config.convert_to_tensor
#         )
#         return embeddings

# def load_embedding_model(config_path: str):
#     with open(config_path, "r") as f:
#         config_dict = json.load(f)
    
#     try:
#         config = SentenceEmbeddingConfig(**config_dict)
#     except TypeError as e:
#         raise ValueError(f"Invalid configuration keys: {e}.")

#     model = SentenceEmbeddingModel(config)
#     return model

In [2]:
config_path = "../config/e5_large_instruct_config.json"

print("Loading E5 Large model...")
embedding_model = load_embedding_model(config_path)
print(f"E5 Large model loaded successfully on {embedding_model.device}!")

Loading E5 Large model...
E5 Large model loaded successfully on cuda!


In [3]:
df = pd.read_csv('../data/cleaned_test.csv')
unique_classes = df['class'].unique().tolist()

item_embeddings = embedding_model.get_embeddings(df['cleaned_text'].tolist(), "query")
class_embeddings = embedding_model.get_embeddings(unique_classes)

# Convert to lists for CSV storage
item_emb_list = item_embeddings.float().cpu().numpy().tolist()
class_emb_list = class_embeddings.float().cpu().numpy().tolist()

# Create class embedding mapping
class_emb_dict = dict(zip(unique_classes, class_emb_list))

df['item_embedding'] = item_emb_list
df['class_embedding'] = [class_emb_dict[cls] for cls in df['class']]

# Create separate columns for unique classes (only for first N rows where N = number of unique classes)
df['unique_classes'] = None
df['unique_classes_embeddings'] = None

# Fill only the first N rows with unique class data
for i, (cls, emb) in enumerate(zip(unique_classes, class_emb_list)):
    if i < len(df):
        df.at[i, 'unique_classes'] = cls
        df.at[i, 'unique_classes_embeddings'] = str(emb)  # Convert list to string for CSV storage

df.to_csv('../results/embeddings.csv', index=False)
print(f"Embeddings saved with {len(df)} rows and {len(unique_classes)} unique classes")

Embeddings saved with 4546 rows and 32 unique classes


In [4]:
df.head(5).to_csv('../results/embeddings_sample.csv', index=False, encoding='utf-8')
print(f"Embeddings saved with {len(df)} rows and {len(unique_classes)} unique classes")

Embeddings saved with 4546 rows and 32 unique classes


In [5]:
load_dotenv('../.env')

TD_HOST = os.getenv('TD_HOST')
TD_USER = os.getenv('TD_USER')
TD_PASS = os.getenv('TD_PASS')
TD_DB = os.getenv('TD_DB')

In [6]:
print(TD_DB,TD_HOST,TD_PASS,TD_USER)
conn = teradatasql.connect(
    host=TD_HOST,
    user=TD_USER,
    password=TD_PASS,
    database=TD_DB
)

print("Successfully connected to Teradata!")

DEMO_USER productclass-goa68jucyz3zqdmo.env.clearscape.teradata.com veles123 demo_user
Successfully connected to Teradata!


In [13]:
# The SQL query to list all databases you have access to
sql_query = "SELECT DatabaseName FROM DBC.DatabasesV ORDER BY DatabaseName;"

print("Listing all databases you have permission to view...")
print("-" * 50)

try:
    cursor = conn.cursor()
    cursor.execute(sql_query)
    
    # Fetch all the results into a list
    databases = cursor.fetchall()

    if not databases:
        print("Could not retrieve a list of databases, or you may not have permission to view any.")
    else:
        print("Here are the databases you can access:")
        for i, (db_name,) in enumerate(databases, 1):
            # .strip() is used to remove any potential whitespace from the result
            print(f"{i}. {db_name.strip()}")

except Exception as e:
    print(f"An error occurred: {e}")

Listing all databases you have permission to view...
--------------------------------------------------
Here are the databases you can access:
1. All
2. Crashdumps
3. DBC
4. dbcmngr
5. Default
6. DemoNow_Monitor
7. demo_user
8. External_AP
9. EXTUSER
10. GLOBAL_FUNCTIONS
11. gs_tables_db
12. LockLogShredder
13. mldb
14. modelops
15. PUBLIC
16. SQLJ
17. SysAdmin
18. SYSBAR
19. SYSJDBC
20. SYSLIB
21. SYSSPATIAL
22. system
23. SystemFe
24. SYSUDTLIB
25. SYSUIF
26. Sys_Calendar
27. tapidb
28. TDaaS_BAR
29. TDaaS_DB
30. TDaaS_Maint
31. TDaaS_Monitor
32. TDaaS_Support
33. TDaaS_TDBCMgmt1
34. TDaaS_TDBCMgmt2
35. TDBCMgmt
36. TDMaps
37. TDPUSER
38. TDQCD
39. TDStats
40. tdwm
41. TD_ANALYTICS_DB
42. TD_SERVER_DB
43. TD_SYSFNLIB
44. TD_SYSGPL
45. TD_SYSXML
46. val


In [15]:
cursor.execute(f"SELECT TOP 1 * FROM {TD_DB}.train_embeddings_fc")
table_columns = [desc[0] for desc in cursor.description]
print("Table columns:", table_columns)

Table columns: ['row_id', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21', 'v22', 'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'v29', 'v30', 'v31', 'v32', 'v33', 'v34', 'v35', 'v36', 'v37', 'v38', 'v39', 'v40', 'v41', 'v42', 'v43', 'v44', 'v45', 'v46', 'v47', 'v48', 'v49', 'v50', 'v51', 'v52', 'v53', 'v54', 'v55', 'v56', 'v57', 'v58', 'v59', 'v60', 'v61', 'v62', 'v63', 'v64', 'v65', 'v66', 'v67', 'v68', 'v69', 'v70', 'v71', 'v72', 'v73', 'v74', 'v75', 'v76', 'v77', 'v78', 'v79', 'v80', 'v81', 'v82', 'v83', 'v84', 'v85', 'v86', 'v87', 'v88', 'v89', 'v90', 'v91', 'v92', 'v93', 'v94', 'v95', 'v96', 'v97', 'v98', 'v99', 'v100', 'v101', 'v102', 'v103', 'v104', 'v105', 'v106', 'v107', 'v108', 'v109', 'v110', 'v111', 'v112', 'v113', 'v114', 'v115', 'v116', 'v117', 'v118', 'v119', 'v120', 'v121', 'v122', 'v123', 'v124', 'v125', 'v126', 'v127', 'v128', 'v129', 'v130', 'v131', 'v132', 'v133', 'v134', 'v135', 'v

In [16]:
df = pd.read_csv('../results/embeddings.csv', encoding='utf-8')

print(f"✅ Loaded CSV with {len(df)} rows")
print(f"Columns: {list(df.columns)}")
print(f"Sample of item_embedding column:")
print(df['item_embedding'].iloc[0][:100] + "...")

✅ Loaded CSV with 4546 rows
Columns: ['class', 'removed_punctuations', 'removed_numbers', 'removed_brand', 'removed_pack', 'removed_unit', 'removed_numbers_and_punctuations', 'removed_stopwords', 'cleaned_text', 'label', 'ara_class', 'item_embedding', 'class_embedding', 'unique_classes', 'unique_classes_embeddings']
Sample of item_embedding column:
[0.00963592529296875, 0.02484130859375, 0.0007939338684082031, -0.056121826171875, 0.011871337890625...


In [17]:
# Parse the embedding strings to actual lists/arrays
def parse_embedding(embedding_str):
    """Convert string representation of list to actual numpy array"""
    try:
        # Use ast.literal_eval to safely parse the string representation
        embedding_list = ast.literal_eval(embedding_str)
        return np.array(embedding_list, dtype=np.float32)
    except:
        print(f"Error parsing embedding: {embedding_str[:50]}...")
        return None

# Apply parsing to item_embedding column
print("🔄 Parsing embeddings...")
df['embedding'] = df['item_embedding'].apply(parse_embedding)

# Remove rows where parsing failed
df = df.dropna(subset=['embedding'])

# Add embedding dimension column
df['embedding_dim'] = df['embedding'].apply(len)

# Create a row_id column (auto-incrementing)
df['row_id'] = range(1, len(df) + 1)

# Select only necessary columns for parquet
parquet_df = df[['row_id', 'embedding', 'embedding_dim']].copy()

# Save to parquet with UTF-8 encoding preserved
parquet_df.to_parquet('full_embeddings.parquet', index=False)

print(f"✅ Saved {len(parquet_df)} rows to gpc_bricks_embeddings.parquet")
print(f"Embedding dimensions: {parquet_df['embedding_dim'].iloc[0]}")

🔄 Parsing embeddings...
✅ Saved 4546 rows to gpc_bricks_embeddings.parquet
Embedding dimensions: 1024


In [18]:
TABLE = f"{TD_DB}.train_embeddings_fc"
print(f"Target table: {TABLE}")

Target table: demo_user.train_embeddings_fc


In [20]:
# Read the parquet file we just created
PARQUET = "full_embeddings.parquet"
df_parquet = pd.read_parquet(PARQUET)

# Get embedding dimension
dim = int(df_parquet["embedding_dim"].iloc[0])
print(f"Embedding dimension: {dim}")

# Create column names: row_id + v1, v2, v3, ..., v1024
cols = ", ".join(["row_id"] + [f"v{i}" for i in range(1, dim+1)])

# Create SQL INSERT statement with placeholders
q = f"INSERT INTO {TABLE} ({cols}) VALUES ({', '.join(['?']*(1+dim))})"

print(f"SQL Query template: {q[:100]}...")
print(f"Total columns: {1 + dim}")

Embedding dimension: 1024
SQL Query template: INSERT INTO demo_user.train_embeddings_fc (row_id, v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12...
Total columns: 1025


In [26]:
# Extract embeddings and row_ids
emb = np.vstack(df_parquet["embedding"])
row_ids = df_parquet["row_id"].to_numpy()

# Create rows for insertion (this part was already correct)
rows = []
for i in range(len(df_parquet)):
    row = [int(row_ids[i])] + emb[i].tolist()
    rows.append(tuple(row))

print(f"Prepared {len(rows)} rows for insertion")
print(f"Sample row structure: row_id={rows[0][0]}, first 5 embedding values={rows[0][1:6]}")


# --- THE SMALL MODIFICATION IS HERE ---
# Create a robust query string that explicitly names every column.
# This prevents the "data shift" error.
column_names = ["row_id"] + [f"v{i}" for i in range(1, 1025)]
placeholders = ", ".join(["?"] * len(column_names)) # Creates "?, ?, ?, ..."
q = f"INSERT INTO {TABLE} ({', '.join(column_names)}) VALUES ({placeholders})"
# -----------------------------------------


# Connect to Teradata and insert data
try:
    with teradatasql.connect(host=TD_HOST, user=TD_USER, password=TD_PASS) as con:
        cursor = con.cursor()
        cursor.executemany(q, rows)
        con.commit()  # Ensure data is committed
    
    print(f"✅ Successfully inserted {len(rows)} rows into {TABLE}")
    
except Exception as e:
    print(f"❌ Error inserting data: {str(e)}")

Prepared 4546 rows for insertion
Sample row structure: row_id=1, first 5 embedding values=(0.00963592529296875, 0.02484130859375, 0.0007939338684082031, -0.056121826171875, 0.011871337890625)
✅ Successfully inserted 4546 rows into demo_user.train_embeddings_fc


In [28]:
import pandas as pd

try:
    with teradatasql.connect(host=TD_HOST, user=TD_USER, password=TD_PASS) as con:

        # Use pandas to get the row count
        count = pd.read_sql(f"SELECT COUNT(*) FROM {TABLE}", con).iloc[0, 0]
        print(f"✅ Total rows in {TABLE}: {count}")

        # Use pandas to get and display the top 5 rows
        print(f"\n--- Top 5 Rows from {TABLE} ---")
        df_head = pd.read_sql(f"SELECT TOP 5 * FROM {TABLE}", con)

        # In a Jupyter cell, this will automatically render as a nice table
        display(df_head)

except Exception as e:
    print(f"❌ Error verifying data: {str(e)}")

C:\Users\mk255155\AppData\Local\Temp\ipykernel_36424\2351680036.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count = pd.read_sql(f"SELECT COUNT(*) FROM {TABLE}", con).iloc[0, 0]


✅ Total rows in demo_user.train_embeddings_fc: 4546

--- Top 5 Rows from demo_user.train_embeddings_fc ---


C:\Users\mk255155\AppData\Local\Temp\ipykernel_36424\2351680036.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_head = pd.read_sql(f"SELECT TOP 5 * FROM {TABLE}", con)


,row_id,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v1015,v1016,v1017,v1018,v1019,v1020,v1021,v1022,v1023,v1024
0,1531,0.021561,0.030045,0.005020,-0.040436,0.017609,-0.007465,-0.004505,0.029495,0.012520,...,-0.037567,-0.030930,0.031006,-0.023148,0.025970,0.029770,-0.008179,-0.026596,-0.029648,0.009315
1,1,0.009636,0.024841,0.000794,-0.056122,0.011871,-0.028091,-0.007812,0.018021,0.024918,...,-0.021820,-0.035278,0.018082,-0.029694,0.025330,0.041473,0.009933,-0.014023,-0.031830,0.028305
2,2,-0.004341,0.033661,0.010498,-0.054352,0.018799,-0.003498,-0.011681,0.073547,0.013206,...,-0.037689,-0.049316,0.016037,-0.016037,0.023804,0.034698,-0.007763,-0.028458,-0.051727,0.032593
3,766,-0.011871,0.024658,0.012207,-0.058167,0.019623,0.013252,-0.006432,0.055298,0.027420,...,-0.021515,-0.049561,0.019867,-0.008057,0.008293,0.038025,-0.006077,-0.021301,-0.062927,0.031555
4,767,0.025299,0.016129,0.009903,-0.046570,0.011963,0.001855,-0.008919,0.060791,0.024765,...,-0.056641,-0.032593,0.001007,-0.031525,0.003613,0.038818,-0.016479,0.008171,-0.039856,0.041107


In [29]:
try:
    with teradatasql.connect(host=TD_HOST, user=TD_USER, password=TD_PASS) as con:
        cursor = con.cursor()
        
        # Delete the rows
        cursor.execute(f"DELETE FROM {TABLE}")
        # Or to delete all: cursor.execute(f"DELETE FROM {TABLE}")
        
        con.commit()
        print("✅ Rows deleted successfully")
        
        # Verify deletion
        cursor.execute(f"SELECT COUNT(*) FROM {TABLE}")
        count = cursor.fetchone()[0]
        print(f"Remaining rows: {count}")
        
except Exception as e:
    print(f"❌ Error deleting data: {str(e)}")

✅ Rows deleted successfully
Remaining rows: 0


In [ ]:
TABLE = f"{TD_DB}.train_embeddings_fc"
CSV_FILE = "full_embeddings.csv"  #Temp CSV

#Load and parse embeddings
df = pd.read_csv('../results/embeddings.csv', encoding='utf-8')
df['embedding'] = df['item_embedding'].apply(lambda s: np.array(ast.literal_eval(s), dtype=np.float32))
df = df.dropna(subset=['embedding'])  #Drop failed parses
df['row_id'] = range(1, len(df) + 1)

#Detect dim from first embedding
dim = len(df['embedding'].iloc[0])

#Expand embeddings to columns v1-v{dim}
emb_df = pd.DataFrame(np.vstack(df['embedding']), columns=[f"v{i}" for i in range(1, dim + 1)])
parquet_df = pd.concat([df[['row_id']], emb_df], axis=1)

#Save expanded to temp CSV
parquet_df.to_csv(CSV_FILE, index=False)

In [ ]:
# Connect and load CSV directly
with teradatasql.connect(host=TD_HOST, user=TD_USER, password=TD_PASS) as con:
    cursor = con.cursor()
    
    #Column names and placeholders for insert
    cols = ["row_id"] + [f"v{i}" for i in range(1, dim + 1)]
    placeholders = ", ".join(["?"] * len(cols))
    
    #Direct CSV load syntax
    q = f"{{fn teradata_read_csv({CSV_FILE})}}INSERT INTO {TABLE} ({', '.join(cols)}) VALUES ({placeholders})"
    cursor.execute(q)
    con.commit()

C:\Users\mk255155\AppData\Local\Temp\ipykernel_36424\1885426725.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count_df = pd.read_sql(f"SELECT COUNT(*) AS total_rows, (SELECT TOP 5 * FROM {TABLE}) FROM {TABLE}", con)


DatabaseError: Execution failed on sql 'SELECT COUNT(*) AS total_rows, (SELECT TOP 5 * FROM demo_user.train_embeddings_fc) FROM demo_user.train_embeddings_fc': [Version 20.0.0.34] [Session 1083] [Teradata Database] [Error 6916] TOP N Syntax error: Top N option is not supported in subquery.
 at gosqldriver/teradatasql.formatError ErrorUtil.go:83
 at gosqldriver/teradatasql.(*teradataConnection).formatDatabaseError ErrorUtil.go:190
 at gosqldriver/teradatasql.(*teradataConnection).makeChainedDatabaseError ErrorUtil.go:206
 at gosqldriver/teradatasql.(*teradataConnection).processErrorParcel TeradataConnection.go:340
 at gosqldriver/teradatasql.(*TeradataRows).processResponseBundle TeradataRows.go:2597
 at gosqldriver/teradatasql.(*TeradataRows).executeSQLRequest TeradataRows.go:1073
 at gosqldriver/teradatasql.newTeradataRows TeradataRows.go:792
 at gosqldriver/teradatasql.(*teradataStatement).QueryContext TeradataStatement.go:122
 at gosqldriver/teradatasql.(*teradataConnection).QueryCon

In [ ]:
# Verify
with teradatasql.connect(host=TD_HOST, user=TD_USER, password=TD_PASS) as con:
    count = pd.read_sql(f"SELECT COUNT(*) FROM {TABLE}", con).iloc[0, 0]
    df_head = pd.read_sql(f"SELECT TOP 5 * FROM {TABLE}", con)
    print(f"Total rows: {count}")
    display(df_head)

C:\Users\mk255155\AppData\Local\Temp\ipykernel_36424\249586929.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count = pd.read_sql(f"SELECT COUNT(*) FROM {TABLE}", con).iloc[0, 0]
C:\Users\mk255155\AppData\Local\Temp\ipykernel_36424\249586929.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_head = pd.read_sql(f"SELECT TOP 5 * FROM {TABLE}", con)


Total rows: 4546


,row_id,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v1015,v1016,v1017,v1018,v1019,v1020,v1021,v1022,v1023,v1024
0,897,0.001010,0.042328,-0.011787,-0.052612,0.027039,-0.012665,-0.013084,0.027130,0.024170,...,-0.004723,-0.020020,0.020584,-0.017120,0.029510,0.053497,0.042908,-0.012344,-0.036041,0.016281
1,898,0.024277,0.036102,-0.003035,-0.051208,0.023453,-0.027725,-0.014931,0.026108,0.030502,...,-0.043060,-0.035400,0.023773,-0.023865,0.029114,0.029922,0.016632,-0.008301,-0.063538,0.034332
2,3047,0.006237,0.025635,-0.007580,-0.035522,0.033752,-0.010620,-0.010368,0.068848,0.023773,...,-0.039948,-0.079590,0.027298,-0.018555,0.017532,0.039459,-0.016983,-0.029114,-0.042450,0.014992
3,359,0.045105,0.036560,0.005260,-0.037140,0.007042,-0.011795,0.003223,0.056671,0.037781,...,-0.029846,-0.022995,0.025146,-0.019196,0.029068,0.041687,0.020599,-0.001346,-0.015404,0.037048
4,360,0.033417,0.035736,-0.022430,-0.048645,0.015411,-0.018021,-0.016113,0.043854,0.023788,...,-0.019760,-0.023331,0.017365,-0.021805,0.017639,0.058136,0.015213,0.015266,-0.007645,0.040131
